In [2]:
import os
while 'notebooks' in os.getcwd():
    os.chdir('..')
    
from dotenv import load_dotenv
load_dotenv()

True

In [47]:
from typing import Union

import requests

from openai import OpenAI
from googlesearch import search
from bs4 import BeautifulSoup
from pydantic import BaseModel

In [87]:
urls = list(search("ufc 300", num_results=10, region="Poland"))

In [88]:
url = urls[0]
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, "html.parser")

In [92]:
text = soup.body.get_text().strip()
# print(text[:1000])

In [82]:
def parse_url(url: str):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    if soup.body is None:
        print(f"Could not find a body for url: {url}")
        return None
    return soup.body.get_text().strip()


def get_websites(query: str, num_results: int = 10):
    for url in search(query, num_results=num_results):
        parsed_text = parse_url(url)
        if parsed_text is None:
            continue
        yield parsed_text

In [67]:
class LLM:
    def __init__(self, system_prompt: str = None, model: str = 'gpt-4.1', response_format=None):
        self.client = OpenAI()
        self.system_prompt = system_prompt
        self.model = model
        self.response_format = response_format
    
    def __call__(self, message: str):
        messages = []
        if self.system_prompt:
            messages.append({"role": "system", "content": self.system_prompt})
        messages.append({"role": "user", "content": message})
        if self.response_format:
            response = self.client.chat.completions.parse(messages=messages, model=self.model, response_format=self.response_format)
            return response.choices[0].message.parsed
        else:
            response = self.client.chat.completions.create(messages=messages, model=self.model)
            return response.choices[0].message.content

In [93]:
class ResponseModel(BaseModel):
    items: list[str]
    
    
llm = LLM("Given a keyword provided by a user, please provide 10 queries to a Google search engine about that subject. Return responses in Italian", response_format=ResponseModel)

In [94]:
queries = llm("cooking").items
queries

['Ricette di cucina facili e veloci',
 'Consigli per cucinare piatti salutari',
 'Tecniche base di cottura in cucina',
 'Come imparare a cucinare da zero',
 'Corsi di cucina online gratuiti',
 'Ricette tradizionali italiane',
 'Utensili indispensabili in cucina',
 'Segreti degli chef per una cucina perfetta',
 'Cucina vegetariana: idee e ricette',
 'Come preparare dolci fatti in casa']

In [95]:
query = queries[0]
text = next(get_websites(query))

In [96]:
cleaner_llm = LLM("Given a raw parsed text from a website, return its cleaned-up version. Returned only the cleaned up version")

In [97]:
clean_text = cleaner_llm(text)

In [98]:
print(clean_text)

Ricette facili e veloci

Nella vostra personale top list delle ricette facili e veloci cosa ci mettereste? Noi abbiamo selezionato un po’ di idee, quelle che funzionano di più a tavola e con gli amici. Magari riuscirete ad organizzare tutto un menù, partendo dall’antipasto finger di bufala e gamberi. Stimolare il palato è essenziale per il buon appetito e se si può giocare con i colori perché non farlo? Le tagliatelle fave e stracciatella manderanno in visibilio le vostre percezioni. Con un po’ di cuore, passione e divertimenti si possono preparare ricette sfiziose come gli straccetti di manzo glassati al balsamico. E per concludere il dolce presto fatto: soltanto 10 minuti per prepararlo e anche meno per finirlo… siamo pronti a scommettere!

Ricette veloci con la pasta

Amanti delle ricette di pasta veloci, affilate le forchette: stanno uscendo dalla nostra cucina idee sfiziose e saporite! Cominciamo dal più grande classico che abbiamo preferito rendere più profumato e saporito: la pa

In [113]:
phrase_llm = LLM("You are a tutor for an Italian language. Given a text, return a list of useful phrases for a language learner. Return exact phrases from the text, without any modifications (including puctuation)", response_format=ResponseModel)
phrases = phrase_llm(text)

In [114]:
len(phrases.items)

55

In [115]:
sum(phrase in text for phrase in phrases.items)

55

In [117]:
def get_neighbouring_context(text, substring, context_size=5):
    index = text.find(substring)
    if index == -1:
        return None  # substring not found
    
    start = max(0, index - context_size)
    end = min(len(text), index + len(substring) + context_size)
    
    return text[start:end]


In [123]:
print(phrases.items[0])
# print(get_neighbouring_context(text, phrases.items[0], 100))

Ricette facili e veloci


In [126]:
translator_llm = LLM("Given a text in Italian, translate it to English.")

In [130]:
translated_phrase = translator_llm(phrases.items[0])
translated_phrase

'Easy and quick recipes'

In [138]:
tutor_llm = LLM(f"The user provides you with a phrase in Italian and its proposed English translation. Explain the phrase and give alternative if possible. Return just the explanation and alternatives")

In [139]:
tutor_llm(f"English: {translated_phrase} \n\n --- \n\n Italian: {phrases.items[0]}")

'The Italian phrase "Ricette facili e veloci" translates directly to "Easy and quick recipes" in English. It is used to describe recipes that are both simple to prepare ("facili") and fast to make ("veloci"). \n\n**Alternatives:**\n- Ricette rapide e semplici\n- Ricette veloci e semplici\n- Piatti facili e veloci\n\nAll of these convey the idea of recipes that do not require much time or complexity.'